# NIS800-53 analysis based on the new NLP based Related Controls

In [1]:

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

import networkx as nx

import google.generativeai as genai
from google.api_core import retry

import os
import time

# import all the helping functions that were written in the base NIS800-53 analysis notebook
from helping_functions import *


# Read the NIS800-53 regulations with the new NLP-based related controls

In [2]:

# NIS_regulations = pd.read_excel("regulations//sp800-53r5-control-catalog.xlsx")
# NIS_regulations = pd.read_feather("regulations//Active_NIS_regulations_with_new_related_controls.feather")
NIS_regulations = pd.read_feather("regulations//Active_NIS_regulations_with_new_related_controls2.feather")
# NIS_regulations = pd.read_parquet("regulations//Active_NIS_regulations_with_new_related_controls.parquet")
NIS_regulations

# remove every key with None value for the new relations columns and sort them from highest to lowest

NIS_regulations.loc[:, "bm25 Related Controls"] = NIS_regulations.loc[:, "bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))
NIS_regulations.loc[:, "normalized bm25 Related Controls"] = NIS_regulations.loc[:, "normalized bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))

NIS_regulations.loc[:, "SentenceTransformer Related Controls"] = NIS_regulations.loc[:, "SentenceTransformer Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))

NIS_regulations.loc[:, "SASS Related Controls"] = NIS_regulations.loc[:, "SASS Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None}).apply(lambda x: dict(sorted(x.items(), key=lambda item: item[1], reverse=True)))
# NIS_regulations.loc[:, "bm25 Related Controls"] = NIS_regulations.loc[:, "bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})
# NIS_regulations.loc[:, "normalized bm25 Related Controls"] = NIS_regulations.loc[:, "normalized bm25 Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})
# NIS_regulations.loc[:, "SentenceTransformer Related Controls"] = NIS_regulations.loc[:, "SentenceTransformer Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})
# NIS_regulations.loc[:, "SASS Related Controls"] = NIS_regulations.loc[:, "SASS Related Controls"].apply(lambda x: {k: v for k, v in x.items() if v is not None})

NIS_regulations


,Control Identifier,Control (or Control Enhancement) Name,Control Text,Discussion,Related Controls,Main Control Name,Family Name,Related Controls Dictionary,Full Text,bm25 Related Controls,normalized bm25 Related Controls,SentenceTransformer Related Controls,SASS Related Controls
0,AC-1,Policy and Procedures,"a. Develop, document, and disseminate to [Assi...",Access control policy and procedures address t...,"IA-1, PM-9, PM-24, PS-8, SI-12 .",AC-1,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...","Policy and Procedures\n\na. Develop, document,...","{'MA-1': 59.27978515625, 'PL-1': 59.2797851562...","{'MA-1': 0.8947177027189795, 'PL-1': 0.8947177...","{'PS-1': 0.9554650783538818, 'CA-1': 0.9547804...","{'CA-1': 0.9555830106820367, 'SC-1': 0.9506337..."
1,AC-2,Account Management,a. Define and document the types of accounts a...,Examples of system account types include indiv...,"AC-3, AC-5, AC-6, AC-17, AC-18, AC-20, AC-24, ...",AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management\n\na. Define and document t...,"{'AC-2(3)': 63.61791229248047, 'AC-2(7)': 62.1...","{'AC-2(3)': 0.5456564097423026, 'AC-2(7)': 0.5...","{'AC-2(7)': 0.8722906112670898, 'AC-2(11)': 0....","{'AC-2(7)': 0.8690222363681739, 'AC-2(11)': 0...."
2,AC-2(1),Account Management | Automated System Account ...,Support the management of system accounts usin...,Automated system account management includes u...,None.,AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management | Automated System Account ...,{},"{'AC-2(2)': 0.6856685718365872, 'AC-2(8)': 0.6...","{'AC-2(8)': 0.8399572968482971, 'IR-4(1)': 0.8...","{'AC-2(8)': 0.8385143606316967, 'AC-2(11)': 0...."
3,AC-2(2),Account Management | Automated Temporary and E...,Automatically [Selection: remove; disable] tem...,Management of temporary and emergency accounts...,None.,AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management | Automated Temporary and E...,{},"{'AC-2(4)': 0.4500737751259309, 'AC-2(3)': 0.4...","{'AC-2(3)': 0.8158748745918274, 'AC-2(4)': 0.8...","{'AC-2(3)': 0.8327054165938192, 'AC-2(13)': 0...."
4,AC-2(3),Account Management | Disable Accounts,Disable accounts within [Assignment: organizat...,"Disabling expired, inactive, or otherwise anom...",None.,AC-2,AC,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Account Management | Disable Accounts\n\nDisab...,"{'AC-2(13)': 23.608457565307617, 'AC-2(2)': 20...","{'AC-2(13)': 0.5667917968522473, 'AC-2(2)': 0....","{'AC-2(13)': 0.8616950511932373, 'AC-2(5)': 0....","{'AC-2(13)': 0.8635755082149525, 'AC-2(2)': 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184,SR-11,Component Authenticity,a. Develop and implement anti-counterfeit poli...,Sources of counterfeit components include manu...,"PE-3, SA-4, SI-7, SR-9, SR-10.",SR-11,SR,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Component Authenticity\n\na. Develop and imple...,"{'SR-11(1)': 28.72646141052246, 'SR-11(3)': 26...","{'SR-11(1)': 0.6496216599515963, 'SR-11(3)': 0...","{'SR-11(1)': 0.8409212231636047, 'SR-11(3)': 0...","{'SR-11(1)': 0.8345096504298886, 'CM-8(3)': 0...."
1185,SR-11(1),Component Authenticity | Anti-counterfeit Trai...,Train [Assignment: organization-defined person...,None.,AT-3.,SR-11,SR,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Component Authenticity | Anti-counterfeit Trai...,{},"{'SR-11': 0.6172898806384338, 'SR-11(3)': 0.54...","{'SR-11(3)': 0.8443235158920288, 'SR-11': 0.84...","{'SR-11(3)': 0.8393997564526279, 'SR-11': 0.83..."
1186,SR-11(2),Component Authenticity | Configuration Control...,Maintain configuration control over the follow...,None.,"CM-3, MA-2, MA-4, SA-10.",SR-11,SR,"{'AC-1': None, 'AC-10': None, 'AC-11': None, '...",Component Authenticity | Configuration Control...,{},"{'SA-4(5)': 0.37485503792576363, 'MA-2': 0.356...","{'MA-2': 0.8005739450454712, 'CM-8(2)': 0.7839...","{'SR-4': 0.8314977910120207, 'MA-2': 0.8313986..."
1187,SR-11(3),Component Authenticity |

In [3]:

family_column = "Family Name"

# group by family name and count the number of regulations in each family
NIS_regulations.groupby(by=[family_column]).apply(len,include_groups=False).mean()
# now interquntile



50.35

In [4]:

NIS_regulations.loc[1,"normalized bm25 Related Controls"]
# NIS_regulations.loc[1,["bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SASS Related Controls"]]
# NIS_regulations.loc[0,"bm25 Related Controls"].keys()
# type(NIS_regulations.loc[0,"bm25 Related Controls"])

# print all key value pairs in NIS_regulations.loc[1,"normalized bm25 Related Controls"]
for k, v in NIS_regulations.loc[1,"normalized bm25 Related Controls"].items():
    print(f"{k}: {v}")
    print("-"*80)
    




AC-2(3): 0.5456564097423026
--------------------------------------------------------------------------------
AC-2(7): 0.5332102684763498
--------------------------------------------------------------------------------
AC-2(9): 0.5220074162177839
--------------------------------------------------------------------------------
AC-2(12): 0.5073050386463656
--------------------------------------------------------------------------------
AC-2(2): 0.497907489534503
--------------------------------------------------------------------------------
AC-2(13): 0.4820235309761175
--------------------------------------------------------------------------------
IA-2(5): 0.4703392514575003
--------------------------------------------------------------------------------
IA-5: 0.46017980787930934
--------------------------------------------------------------------------------
AC-2(8): 0.4444947807971446
--------------------------------------------------------------------------------
AC-6(2): 0.441447301

## transform the old Related Controls from the string NIS representation to the new Dictionary representation

In [5]:

# Related_Controls = row["Related Controls"].replace(" .","").replace(".","").split(", ")
# apply the replace and split operation on the Related Controls column to get a list of related controls
# NIS_regulations["Related Controls"].apply(lambda x: {k: v for k, v in (i.split("-") for i in x.replace(" .","").replace(".","").split(", ")) if v is not None else None })
new_NIS_Related_Controls = NIS_regulations["Related Controls"].apply(lambda x: x.replace(" .","").replace(".","").split(", "))
# transform the list of related controls into a dictionary where the key is the control and the value is 0.5
# new_NIS_Related_Controls[3]
# {x: 0.5 for x in new_NIS_Related_Controls[0] if x != "None"}
new_NIS_Related_Controls = new_NIS_Related_Controls.apply(lambda x: {k: 0.5 for k in x if k!= "None"})
NIS_regulations["Related Controls Dictionary"] = new_NIS_Related_Controls

new_NIS_Related_Controls


0       {'IA-1': 0.5, 'PM-9': 0.5, 'PM-24': 0.5, 'PS-8...
1       {'AC-3': 0.5, 'AC-5': 0.5, 'AC-6': 0.5, 'AC-17...
2                                                      {}
3                                                      {}
4                                                      {}
                              ...                        
1184    {'PE-3': 0.5, 'SA-4': 0.5, 'SI-7': 0.5, 'SR-9'...
1185                                        {'AT-3': 0.5}
1186    {'CM-3': 0.5, 'MA-2': 0.5, 'MA-4': 0.5, 'SA-10...
1187                                        {'RA-5': 0.5}
1188                                        {'MP-6': 0.5}
Name: Related Controls, Length: 1007, dtype: object

# Checking the "Related Controls" column

We want to know what are the stats of the current state.

## Run describe on the "Related Controls" column

In [6]:
NIS_regulations.loc[:,["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SASS Related Controls"]].describe(include="all")

,Related Controls Dictionary,bm25 Related Controls,normalized bm25 Related Controls,SentenceTransformer Related Controls,SASS Related Controls
count,1007,1007,1007,1007,1007
unique,540,512,927,961,936
top,{},{},{},{},{}
freq,361,496,81,47,72


We can see that like in the human-made relations there are still some regulations that have no related controls.

## Look if the controls are mutually related
If control A or any of its sub-controls is related to control B, then control B or any of its sub-controls are also related to control A 

In order to help with future operations we will create a function that will get a pandas DataFrame of the regulations with columns indicating the name of the regulation and what family it belongs to and will return a statistical analysis of the inner vs outer connections between regulation families.

In [7]:

# mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,"SASS Related Controls",do_print=True)
NIS_regulations.loc[0,"SASS Related Controls"]

# NIS_regulations.loc[NIS_regulations["Control Identifier"]=='AU-9(6)',"SASS Related Controls"].iloc[0]
# NIS_regulations.loc[NIS_regulations["Control Identifier"]=='AC-1',"SASS Related Controls"].iloc[0]


{'CA-1': 0.9555830106820367,
 'SC-1': 0.9506337540263092,
 'PS-1': 0.950139034532604,
 'IA-1': 0.9433262866960146,
 'SI-1': 0.9429564565962818,
 'AU-1': 0.9407610350576225,
 'PL-1': 0.9373220718620325,
 'RA-1': 0.9359978956442284,
 'CM-1': 0.931933541992275,
 'SA-1': 0.9307049720572087,
 'AT-1': 0.9300006787133108,
 'MP-1': 0.9299980144604996,
 'MA-1': 0.9297985016335292,
 'PE-1': 0.9278253604512812,
 'CP-1': 0.9266664519506163,
 'IR-1': 0.9263953417707502,
 'SR-1': 0.9217372405386126,
 'PT-1': 0.9198076335048287,
 'PM-1': 0.8537291032496708,
 'PS-6': 0.8497113984592382,
 'PM-17': 0.8449044644379837,
 'PM-18': 0.8416750267654811,
 'PL-2': 0.8317797415249137,
 'PL-4': 0.8267371202970233,
 'SA-5': 0.8183020101710419,
 'PM-9': 0.817993217307679,
 'AC-24': 0.8131593206230536,
 'CM-9': 0.8118572691295027,
 'AC-3(15)': 0.8102405585450353,
 'PM-11': 0.8076261905132122,
 'PM-4': 0.8075817547660697,
 'IR-8': 0.8023269446399361,
 'CM-3': 0.8003772396607272,
 'SR-3': 0.7991990343749603,
 'PM-27':

In [8]:

relations_columns = ["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SASS Related Controls"]

for relation_column in relations_columns:
    mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,relation_column,do_print=False)
    print(f'there are {len(not_mutually_related)} non-mutual connections in "{relation_column}"')
    print(f"in-family connections:\n{in_family_connection_statistics}")
    # print(f"Relations statistics:\n{relations_statistics}")
    s_r = sum(mutually_related["same family"]) / len(mutually_related)
    if len(not_mutually_related) > 0:
        s_r_n = sum(not_mutually_related["same family"]) / len(not_mutually_related)
    else:
        s_r_n = 0
    s = (sum(mutually_related["same family"]) + sum(not_mutually_related["same family"])) / (len(mutually_related) + len(not_mutually_related))
    print(f"{s:.2%} of existing connections are in-family")
    print(f"{s_r:.2%} of existing mutual connections are in-family")
    print(f"{s_r_n:.2%} of existing non-mutual connections are in-family")
    print("-"*100)
# mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,"bm25 Related Controls",do_print=True)
# mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,"SentenceTransformer Related Controls",do_print=True)
# mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,"SASS Related Controls",do_print=False)
# print(f"there are {len(not_mutually_related)} non-mutual connections")

mutually_related, not_mutually_related, in_family_connection_statistics, relations_statistics = get_mutual_relations(NIS_regulations,relations_columns[2:],do_print=False)

print(f'there are {len(not_mutually_related)} non-mutual connections in "{relations_columns[2:]}"')
print(f"in-family connections:\n{in_family_connection_statistics}")
# print(f"Relations statistics:\n{relations_statistics}")
s_r = sum(mutually_related["same family"]) / len(mutually_related)
if len(not_mutually_related) > 0:
    s_r_n = sum(not_mutually_related["same family"]) / len(not_mutually_related)
else:
    s_r_n = 0
s = (sum(mutually_related["same family"]) + sum(not_mutually_related["same family"])) / (len(mutually_related) + len(not_mutually_related))
print(f"{s:.2%} of existing connections are in-family")
print(f"{s_r:.2%} of existing mutual connections are in-family")
print(f"{s_r_n:.2%} of existing non-mutual connections are in-family")
print("-"*100)



there are 645 non-mutual connections in "Related Controls Dictionary"
in-family connections:
number of families                                                                                      20.000000
mean number of controls in family                                                                       50.350000
median number of controls in family                                                                     38.500000
std of number of controls in family                                                                     38.785883
Inter-quartile range of number of controls in family                                                    34.250000
number of participating controls                                                                       298.000000
balance score 1 (mean/std)                                                                               1.298153
balance score 2 (median/Inter-quartile range)                                                            1.12

In [9]:
# relations_statistics
# relations_statistics["actual outside relations"]

For the related controls we got from "bm25 Related Controls", we see a lot of non-mutual connections, 2901 compered to 645 from the human-made relations. This may be because the BM25 score function is not commutative so the distance from control A to control B is not the same as from control B to control A. Apart from that, the non-normalized bm25 scores look different from all the other tested methods, and it gives worst statistical indicators then them and the human-made relations.

For the related controls we got from "normalized bm25 Related Controls", we see some non-mutual connections, 1014 compered to 645 from the human-made relations. This is probably due to the score normalization step we used.

For the related controls we got from "SentenceTransformer Related Controls" and "SASS Related Controls" we got 0 non-mutual connections. This is because the scores from them are normalized by default and because that they are commutative so the distance from control A to control B is the same as from control B to control A.

Furthermore (Apart from the "bm25 Related Controls" scores), the statistics of the in-family connections are all higher than what we got from the human-made relations. 

# Make a network graph based on the "Related Controls" column


In [10]:

# relations_columns = ["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SASS Related Controls"]

# is_directed:bool = False
# is_Main_Controls_Only:bool = True
# is_Weighted:bool = True

# Related_Controls_Graph = make_network_graph_from_relations(NIS_regulations, relations_columns[2:], 
#                                                            is_directed=is_directed, is_Main_Controls_Only=is_Main_Controls_Only, is_Weighted=is_Weighted)

# connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=True, draw_spring=False)

# in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(connected_components[0]),
#                                                                                                                NIS_regulations,relations_columns[2:],Add_Weights=is_Weighted,Only_Main_Controls=is_Main_Controls_Only)



In [11]:

relations_columns = ["Related Controls Dictionary","bm25 Related Controls","normalized bm25 Related Controls","SentenceTransformer Related Controls","SASS Related Controls"]

is_directed:bool = False
is_Main_Controls_Only:bool = True
is_Weighted:bool = True

add_groups_summaries:bool = True

i:int=1
Relations = relations_columns[0]

Related_Controls_Graph = make_network_graph_from_relations(NIS_regulations, Relations, 
                                                           is_directed=is_directed, is_Main_Controls_Only=is_Main_Controls_Only, is_Weighted=is_Weighted)
if is_directed:
    strongly_connected_components, weakly_connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=False)
    in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(weakly_connected_components[0]),
                                                                                  NIS_regulations,Relations,Add_Weights=is_Weighted,Only_Main_Controls=is_Main_Controls_Only,
                                                                                  add_groups_summaries=add_groups_summaries,
                                                                                  do_Modularity_based_communities=True, do_Louvain_communities=True, do_Fluid_communities=True,
                                                                                  do_Divisive_communities=False, do_Label_propagation_communities=True, do_Centrality_communities=True,
                                                                                  )

else:
    connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=False)
    in_family_connection_statistics_test, relations_DataFrames = cluster_analysis(Related_Controls_Graph.subgraph(connected_components[0]),
                                                                                  NIS_regulations,Relations,Add_Weights=is_Weighted,Only_Main_Controls=is_Main_Controls_Only,
                                                                                  add_groups_summaries=add_groups_summaries,
                                                                                  do_Modularity_based_communities=True, do_Louvain_communities=True, do_Fluid_communities=True,
                                                                                  do_Divisive_communities=False, do_Label_propagation_communities=True, do_Centrality_communities=True,
                                                                                  )

    in_family_connection_statistics_test





number of nodes in graph: 298
number of edges in graph: 0
number of nodes in graph: 298
number of edges in graph: 0
number of nodes in graph: 298
number of edges in graph: 1906
number of connected components: 6
component 0 has 293 nodes
component 1 has 1 nodes
{'AT-6'}
component 2 has 1 nodes
{'MP-8'}
component 3 has 1 nodes
{'SC-51'}
component 4 has 1 nodes
{'SI-22'}
component 5 has 1 nodes
{'SI-23'}
----------------------------------------------------------------------------------------------------
base NIS800-53 families:
the in_family_connection_ratios are: 0.248 , 0.232
number of families                                                                                      20.000000
mean number of controls in family                                                                       50.350000
median number of controls in family                                                                     38.500000
std of number of controls in family                                         

In [12]:


with pd.ExcelWriter('regulations outputs\\in family connection statistics test old.xlsx') as writer:  
    in_family_connection_statistics_test.to_excel(writer, sheet_name="methods comparison",merge_cells=False,freeze_panes=(1,1))
    for i in range(len(relations_DataFrames)) :
        relations_DataFrames[i].to_excel(writer, sheet_name=relations_DataFrames[i].index.name, merge_cells=False, freeze_panes=(1,1))
    # NIS_relations_statistics.to_excel(writer, sheet_name="original NIS statistics",merge_cells=False,freeze_panes=(1,1))
    # Modularity_based_communities_relations_statistics.to_excel(writer, sheet_name="Modularity statistics",merge_cells=False,freeze_panes=(1,1))
    # Louvain_Communities_relations_statistics.to_excel(writer, sheet_name="Louvain statistics",merge_cells=False,freeze_panes=(1,1))
    # Fluid_Communities_relations_statistics.to_excel(writer, sheet_name="Fluid statistics",merge_cells=False,freeze_panes=(1,1))

in_family_connection_statistics_test


,NIS800-53 families,Modularity based communities,Louvain Communities,Fluid Communities,Label propagation,Centrality Communities
number of families,20.000000,7.000000,7.000000,20.000000,2.000000,20.000000
mean number of controls in family,50.350000,142.571429,142.571429,49.900000,499.000000,49.900000
median number of controls in family,38.500000,102.000000,156.000000,37.000000,499.000000,1.000000
std of number of controls in family,38.785883,139.070554,77.330152,31.328312,701.449927,215.160136
Inter-quartile range of number of controls in family,34.250000,246.000000,95.500000,33.000000,496.000000,2.000000
number of participating controls,298.000000,293.000000,293.000000,293.000000,293.000000,293.000000
balance score 1 (mean/std),1.298153,1.025173,1.843672,1.592808,0.711384,0.231920
balance score 2 (median/Inter-quartile range),1.124088,0.414634,1.633508,1.121212,1.006048,0.500000
same family connections,785.000000,1812.000000,1619.000000,1133.000000,3165.000000,3109.000000
out of family connections,2382.000000,1355.000000,1548.000000,2034.000000,2.000000,58.000000


In [13]:


relations_DataFrames[0].index.name


'original NIS families'

In [16]:

len(relations_DataFrames)
relations_DataFrames[2]
# relations_DataFrames[2].loc[:,["actual inside relations","actual outside relations"]]
# NIS_regulations["Main Control Name"].isin(["SC-20", "SC-21", "SC-22" ])
# NIS_regulations.loc[NIS_regulations["Main Control Name"].isin(["SC-20", "SC-21", "SC-22" ]),:]
# NIS_regulations.loc[NIS_regulations["Main Control Name"].isin(["SA-9" ]),:]
# NIS_regulations.loc[NIS_regulations["Main Control Name"].isin(["SC-20", "SC-21", "SC-22" ]),relations_columns[i]]
# [SC-20, SC-21, SC-22]
# connected_components = inspect_connected_components(Related_Controls_Graph, draw_kamada_kawai=True)
# Related_Controls_Graph["SC-22"]
# Related_Controls_Graph["SA-9"]
# Figure = plt.figure(figsize=(30,30))
# nx.draw_kamada_kawai(Related_Controls_Graph, with_labels=True, font_weight='bold')
# nx.draw_spring(Related_Controls_Graph, with_labels=True, font_weight='bold', edge_color='purple')
i=0
relations_DataFrames[1].loc[i,"names of regulations"]
group = NIS_regulations["Main Control Name"].isin(relations_DataFrames[1].loc[i,"names of regulations"])
NIS_regulations.loc[group]
# NIS_regulations.loc[group,"Full Text"]

# print(NIS_regulations.loc[group,"Full Text"])
regulations_examples:str = ""
for i,regulations_text in enumerate(NIS_regulations.loc[group,"Full Text"]):
    regulations_examples += f"Regulation {i+1}:\n{regulations_text}\n\n"

    # break
print(regulations_examples)


Regulation 1:
Concurrent Session Control

Limit the number of concurrent sessions for each [Assignment: organization-defined account and/or account type] to [Assignment: organization-defined number].

Regulation 2:
Access Control for Mobile Devices

a. Establish configuration requirements, connection requirements, and implementation guidance for organization-controlled mobile devices, to include when such devices are outside of controlled areas; and
b. Authorize the connection of mobile devices to organizational systems.

Regulation 3:
Access Control for Mobile Devices | Restrictions for Classified Information

(a) Prohibit the use of unclassified mobile devices in facilities containing systems processing, storing, or transmitting classified information unless specifically permitted by the authorizing official; and
(b) Enforce the following restrictions on individuals permitted by the authorizing official to use unclassified mobile devices in facilities containing systems processing, s

In [17]:

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

for model in genai.list_models():
  # print(model.name)
  # print(model.supported_generation_methods)
  if 'generateContent' in model.supported_generation_methods:
    print(model.name)



models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.0-flash-exp
models/gemini-exp-1206
models/gemini-exp-1121
models/gemini-exp-1114
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-experimental


In [18]:

# genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
# system_instruction:str = "You are an expert in cyber regulations. \
#   Your task is to read all the regulations you are given and summarize them in up to 5 short and concise bullet points. \
#   In your answers give the general topic that represents all of the regulations you were given."
# model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp",system_instruction=system_instruction)
# retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}

# response = model.generate_content(f"Please summarize this group of cyber regulations:\n\n{regulations_examples}",
#             request_options=retry_policy,)

# response




In [19]:

# print(response.text)


In [20]:

# # add summarizion to the groups of regulations
# def add_summation(regulations_DataFrame:pd.DataFrame, relations_DataFrame:pd.DataFrame, is_Main_Controls_Only:bool=True) -> pd.DataFrame:

#     for inx in relations_DataFrame.index:
#         # get the relevant regulations
#         if is_Main_Controls_Only:
#             group = regulations_DataFrame["Main Control Name"].isin(relations_DataFrame.loc[inx,"names of regulations"])
#         else:
#             group = regulations_DataFrame["Control Identifier"].isin(relations_DataFrame.loc[inx,"names of regulations"])
#         regulations_examples:str = ""
#         for i,regulations_text in enumerate(NIS_regulations.loc[group,"Full Text"]):
#             regulations_examples += f"Regulation {i+1}:\n{regulations_text}\n\n"

#             # break
#         # print(regulations_examples)

#         # summarize the regulations
#         system_instruction:str = "You are an expert in cyber regulations. \
#         Your task is to read all the regulations you are given and summarize them in up to 5 short and concise bullet points. \
#         In your answers give the general topic that represents all of the regulations you were given."
#         model = genai.GenerativeModel(model_name="gemini-2.0-flash-exp",system_instruction=system_instruction)
#         retry_policy = {"retry": retry.Retry(predicate=retry.if_transient_error)}
#         response = model.generate_content(f"Please summarize this group of cyber regulations:\n\n{regulations_examples}",
#                                           request_options=retry_policy,)
        
#         # add the responses to the summary column in the relations_DataFrame
#         relations_DataFrame.loc[inx,"Summary"] = response.text.strip()

#         time.sleep(6) # sleep for a little bit because of RPM limitations

#     return relations_DataFrame

# relations_DataFrame_with_summary = add_summation(NIS_regulations,relations_DataFrames[1],is_Main_Controls_Only=True)
# relations_DataFrame_with_summary


In [21]:
# # print all the summaries
# for inx in relations_DataFrame_with_summary.index:
#     print(f"Regulation Group {inx+1}:\nSummary: {relations_DataFrame_with_summary.loc[inx,'Summary']}\n\n")
#     print("-"*100)
#     # break




In [22]:

in_family_connection_statistics_test
# relations_DataFrames



,NIS800-53 families,Modularity based communities,Louvain Communities,Fluid Communities,Label propagation,Centrality Communities
number of families,20.000000,7.000000,7.000000,20.000000,2.000000,20.000000
mean number of controls in family,50.350000,142.571429,142.571429,49.900000,499.000000,49.900000
median number of controls in family,38.500000,102.000000,156.000000,37.000000,499.000000,1.000000
std of number of controls in family,38.785883,139.070554,77.330152,31.328312,701.449927,215.160136
Inter-quartile range of number of controls in family,34.250000,246.000000,95.500000,33.000000,496.000000,2.000000
number of participating controls,298.000000,293.000000,293.000000,293.000000,293.000000,293.000000
balance score 1 (mean/std),1.298153,1.025173,1.843672,1.592808,0.711384,0.231920
balance score 2 (median/Inter-quartile range),1.124088,0.414634,1.633508,1.121212,1.006048,0.500000
same family connections,785.000000,1812.000000,1619.000000,1133.000000,3165.000000,3109.000000
out of family connections,2382.000000,1355.000000,1548.000000,2034.000000,2.000000,58.000000


## Look on the connected components in the Related_Controls_Graph

In [23]:

# strongly_connected_components, weakly_connected_components = inspect_connected_components(Related_Controls_Graph)
connected_components = inspect_connected_components(Related_Controls_Graph)
connected_components



number of connected components: 6
component 0 has 293 nodes
component 1 has 1 nodes
{'AT-6'}
component 2 has 1 nodes
{'MP-8'}
component 3 has 1 nodes
{'SC-51'}
component 4 has 1 nodes
{'SI-22'}
component 5 has 1 nodes
{'SI-23'}


[{'AC-1',
  'AC-10',
  'AC-11',
  'AC-12',
  'AC-14',
  'AC-16',
  'AC-17',
  'AC-18',
  'AC-19',
  'AC-2',
  'AC-20',
  'AC-21',
  'AC-22',
  'AC-23',
  'AC-24',
  'AC-25',
  'AC-3',
  'AC-4',
  'AC-5',
  'AC-6',
  'AC-7',
  'AC-8',
  'AC-9',
  'AT-1',
  'AT-2',
  'AT-3',
  'AT-4',
  'AU-1',
  'AU-10',
  'AU-11',
  'AU-12',
  'AU-13',
  'AU-14',
  'AU-16',
  'AU-2',
  'AU-3',
  'AU-4',
  'AU-5',
  'AU-6',
  'AU-7',
  'AU-8',
  'AU-9',
  'CA-1',
  'CA-2',
  'CA-3',
  'CA-5',
  'CA-6',
  'CA-7',
  'CA-8',
  'CA-9',
  'CM-1',
  'CM-10',
  'CM-11',
  'CM-12',
  'CM-13',
  'CM-14',
  'CM-2',
  'CM-3',
  'CM-4',
  'CM-5',
  'CM-6',
  'CM-7',
  'CM-8',
  'CM-9',
  'CP-1',
  'CP-10',
  'CP-11',
  'CP-12',
  'CP-13',
  'CP-2',
  'CP-3',
  'CP-4',
  'CP-6',
  'CP-7',
  'CP-8',
  'CP-9',
  'IA-1',
  'IA-10',
  'IA-11',
  'IA-12',
  'IA-2',
  'IA-3',
  'IA-4',
  'IA-5',
  'IA-6',
  'IA-7',
  'IA-8',
  'IA-9',
  'IR-1',
  'IR-2',
  'IR-3',
  'IR-4',
  'IR-5',
  'IR-6',
  'IR-7',
  'IR-8',
  'IR-9'

In [24]:

type(connected_components[0])
# sort list of sets by size in descending order

sorted_connected_components = sorted(connected_components, key=lambda x: len(x), reverse=True)
sorted_connected_components


[{'AC-1',
  'AC-10',
  'AC-11',
  'AC-12',
  'AC-14',
  'AC-16',
  'AC-17',
  'AC-18',
  'AC-19',
  'AC-2',
  'AC-20',
  'AC-21',
  'AC-22',
  'AC-23',
  'AC-24',
  'AC-25',
  'AC-3',
  'AC-4',
  'AC-5',
  'AC-6',
  'AC-7',
  'AC-8',
  'AC-9',
  'AT-1',
  'AT-2',
  'AT-3',
  'AT-4',
  'AU-1',
  'AU-10',
  'AU-11',
  'AU-12',
  'AU-13',
  'AU-14',
  'AU-16',
  'AU-2',
  'AU-3',
  'AU-4',
  'AU-5',
  'AU-6',
  'AU-7',
  'AU-8',
  'AU-9',
  'CA-1',
  'CA-2',
  'CA-3',
  'CA-5',
  'CA-6',
  'CA-7',
  'CA-8',
  'CA-9',
  'CM-1',
  'CM-10',
  'CM-11',
  'CM-12',
  'CM-13',
  'CM-14',
  'CM-2',
  'CM-3',
  'CM-4',
  'CM-5',
  'CM-6',
  'CM-7',
  'CM-8',
  'CM-9',
  'CP-1',
  'CP-10',
  'CP-11',
  'CP-12',
  'CP-13',
  'CP-2',
  'CP-3',
  'CP-4',
  'CP-6',
  'CP-7',
  'CP-8',
  'CP-9',
  'IA-1',
  'IA-10',
  'IA-11',
  'IA-12',
  'IA-2',
  'IA-3',
  'IA-4',
  'IA-5',
  'IA-6',
  'IA-7',
  'IA-8',
  'IA-9',
  'IR-1',
  'IR-2',
  'IR-3',
  'IR-4',
  'IR-5',
  'IR-6',
  'IR-7',
  'IR-8',
  'IR-9'